In [95]:
from futu import *
import time
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

2021-02-24 11:46:36,511 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2021-02-24 11:46:36,513 [open_context_base.py] on_connected:344: Connected : conn_id=6; 
2021-02-24 11:46:36,516 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=6; info={'server_version': 500, 'login_user_id': 22320258, 'conn_id': 6770187133051955054, 'conn_key': '25029316CE926CD2', 'conn_iv': '7D779310A75B51EF', 'keep_alive_interval': 10, 'is_encrypt': False};


In [96]:
# pip install futu-api # 不能直接 pip install futu

# 目标：

- 存储历史数据
- 获取并下载实时数据，对接到历史数据
- 计算每一只个股到最新数据的动态指标情况

# 个股范围：

- 所有港股
- 所有标普500股票

# API 文档
https://openapi.futunn.com/futu-api-doc/quote/get-stock-filter.html

# ====获取历史 K 线====
https://www.tdameritrade.com/tools-and-platforms/thinkorswim.page

- 1min：K_1M
- 3min: K_3M
- 5min: K_5M
- 15min：K_15M
- 30min: K_30M
- 60min: K_60M
- 1day：K_DAY
- 1week：K_WEEK
- 1month：K_MON
- 1quarter: K_QUARTER
- 1year：K_YEAR

In [97]:
"""
参数设置
"""

end_date = "2021-2-24"

In [98]:
# 定义每个筛选模型

# 1. MACD
def MACD(df, price_col, n_fast, n_slow): # n_fast = 12, n_slow = 26
    """
    http://stockcharts.com/docs/doku.php?id=scans:indicators
    MACD, MACD Signal and MACD difference, rationale CHECKED, code CHECKED, updated
    # Conventional look-back window for calculating MACDsign is 9
    """
    EMAfast = df[price_col].ewm(span = n_fast, min_periods = n_fast - 1).mean()
    EMAslow = df[price_col].ewm(span = n_slow, min_periods = n_slow - 1).mean()
    diff = pd.Series(EMAfast - EMAslow)
    dea = diff.ewm(span = 9, min_periods = 8).mean()
    macd = (pd.Series(diff - dea))*2
    df["DIFF"] = diff
    df["DEA"] = dea
    df["MACD"] = macd
    return df

# 在MACD模型下的开平仓算法

def macd_updown_signals(df, signal_spread_col, date_col, ticker_col):
    ticker = df[ticker_col].values[-1]
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    del df['index']
    listLongShort = []
    listDate = []
    macd = df[signal_spread_col].values
    
    for i in range(1, len(df)):
        last_date = df[date_col][i]
        
        if macd[i]>macd[i-1] and macd[i-1]<macd[i-2]:
            
            listLongShort.append("BUY")
            listDate.append(last_date)
        #                          # The other way around
        elif macd[i]<macd[i-1] and macd[i-1]>macd[i-2]:
            listLongShort.append("SELL")
            listDate.append(last_date)
        #                          # Do nothing if not satisfied
        elif macd[i]<macd[i-1]:
            listLongShort.append("HOLD SHORT")
            listDate.append(last_date)
            
        elif macd[i]>macd[i-1]:
            listLongShort.append("HOLD LONG")
            listDate.append(last_date)           
            
    return ticker, listDate[-1], listLongShort[-1]
#     df['Advice'] = listLongShort
    # The advice column means "Buy/Sell/Hold" at the end of this day or
    #  at the beginning of the next day, since the market will be closed
    
def macd_cross_signals(df, signal_spread_col, date_col, ticker_col):
    ticker = df[ticker_col].values[-1]
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    del df['index']
    listLongShort = []
    listDate = []
    macd = df[signal_spread_col].values
    
    for i in range(1, len(df)):
        last_date = df[date_col][i]
        #                          # If the MACD crosses the signal line upward
        if macd[i] >0 and macd[i - 1] <0:
            listLongShort.append("BUY")
            listDate.append(last_date)
        #                          # The other way around
        elif macd[i] < 0 and macd[i - 1] >0:
            listLongShort.append("SELL")
            listDate.append(last_date)
        #                          # Do nothing if not crossed
        else: # 还要改，再增加点条件
            listLongShort.append("HOLD")
            listDate.append(last_date)
#     print("Ticker: ", ticker)
#     print("Last Date", listDate[-1])
#     print("Last Signal", listLongShort[-1])
    return ticker, listDate[-1], listLongShort[-1]
#     df['Advice'] = listLongShort
    # The advice column means "Buy/Sell/Hold" at the end of this day or
    #  at the beginning of the next day, since the market will be closed

In [99]:
ticker_list = ['HK.09988','HK.06618','HK.03888','HK.00418','HK.09999','HK.01909',
              'HK.01070','HK.06969','HK.00241','HK.01137','HK.06055','HK.06862',
              'HK.00700','HK.02051','HK.06993','HK.06618','HK.09992','HK.03690',
              'HK.01810','HK.01024','HK.01119','HK.08029','HK.06688','HK.00493',
              'HK.00175','HK.06969','HK.01610','HK.02400']

"""
Loading Data from FUTU
"""
def freq_15M_macd_up_down(start_date, current_date):
    all_list = []
    for ticker in ticker_list:
        df_list = []
        ret, data, page_req_key = quote_ctx.request_history_kline(ticker, start=start_date, end=current_date, max_count=500, ktype = KLType.K_15M)  # 每页1000个，请求第一页
        if ret == RET_OK:
    #         print("First Page")
    #         print(data)
            all_list.append(data)
        #     print(data['code'][0])    # 取第一条的股票代码
        #     print(data['close'].values.tolist())   # 第一页收盘价转为list
        else:
            print('error:', data)

    #     while page_req_key != None:  # 请求后面的所有结果
    #         print('*************************************')
    #         ret, data, page_req_key = quote_ctx.request_history_kline('HK.00700', start='2019-09-11', end='2020-09-18', max_count=500, ktype = KLType.K_5M, page_req_key=page_req_key) # 请求翻页后的数据
    #         if ret == RET_OK:
    #             print("==========WHILE==========")
    #             print(data)
    #             df_list.append(data)
    #         else:
    #             print('error:', data)
    # print('All pages are finished!')
    all_df = pd.concat(all_list)


    """
    Calculating Signals
    """

    stock_list = []
    date_list = []
    signal_list = []
    len_ = 1
    date_col = "time_key"
    price_col = "close"
    ticker_col = "code"
    signal_df = pd.DataFrame()
    model_freq = "15M"
    model_name = "Macd_Updown_Signals"
    total_number = len(ticker_list)


    for ticker in ticker_list:
        print ("Calculating Signals for No.{} : {}".format(len_, ticker))
        data = all_df[all_df['code']==ticker]
        try:
            data_macd = MACD(data, price_col, 12, 26)
            ticker, last_date, last_signal = macd_updown_signals(data_macd, "MACD", date_col, ticker_col)
            stock_list.append(ticker)
            date_list.append(last_date)
            signal_list.append(last_signal)
        except Exception as e:
            print(e)
        len_+= 1
    signal_df['Ticker'] = stock_list
    signal_df['Last_Timestamp'] = date_list
    signal_df['Signal'] = signal_list
    signal_df['model_name'] = model_name
    signal_df['model_freq'] = model_freq
    print("All Done!")
    return signal_df


"""
Loading Data from FUTU
"""
def freq_60M_macd_up_down(start_date, current_date):
    all_list = []
    for ticker in ticker_list:
        df_list = []
        ret, data, page_req_key = quote_ctx.request_history_kline(ticker, start=start_date, end=current_date, max_count=500, ktype = KLType.K_60M)  # 每页1000个，请求第一页
        if ret == RET_OK:
    #         print("First Page")
    #         print(data)
            all_list.append(data)
        #     print(data['code'][0])    # 取第一条的股票代码
        #     print(data['close'].values.tolist())   # 第一页收盘价转为list
        else:
            print('error:', data)

    #     while page_req_key != None:  # 请求后面的所有结果
    #         print('*************************************')
    #         ret, data, page_req_key = quote_ctx.request_history_kline('HK.00700', start='2019-09-11', end='2020-09-18', max_count=500, ktype = KLType.K_5M, page_req_key=page_req_key) # 请求翻页后的数据
    #         if ret == RET_OK:
    #             print("==========WHILE==========")
    #             print(data)
    #             df_list.append(data)
    #         else:
    #             print('error:', data)
    # print('All pages are finished!')
    all_df = pd.concat(all_list)


    """
    Calculating Signals
    """

    stock_list = []
    date_list = []
    signal_list = []
    len_ = 1
    date_col = "time_key"
    price_col = "close"
    ticker_col = "code"
    signal_df = pd.DataFrame()
    model_freq = "60M"
    model_name = "Macd_Updown_Signals"
    total_number = len(ticker_list)


    for ticker in ticker_list:
        print ("Calculating Signals for No.{} : {}".format(len_, ticker))
        data = all_df[all_df['code']==ticker]
        try:
            data_macd = MACD(data, price_col, 12, 26)
            ticker, last_date, last_signal = macd_updown_signals(data_macd, "MACD", date_col, ticker_col)
            stock_list.append(ticker)
            date_list.append(last_date)
            signal_list.append(last_signal)
        except Exception as e:
            print(e)
        len_+= 1
    signal_df['Ticker'] = stock_list
    signal_df['Last_Timestamp'] = date_list
    signal_df['Signal'] = signal_list
    signal_df['model_name'] = model_name
    signal_df['model_freq'] = model_freq
    print("All Done!")
    return signal_df

def freq_1D_macd_up_down(start_date, current_date):
    all_list = []
    for ticker in ticker_list:
        df_list = []
        ret, data, page_req_key = quote_ctx.request_history_kline(ticker, start=start_date, end=current_date, max_count=500, ktype = KLType.K_DAY)  # 每页1000个，请求第一页
        if ret == RET_OK:
    #         print("First Page")
    #         print(data)
            all_list.append(data)
        #     print(data['code'][0])    # 取第一条的股票代码
        #     print(data['close'].values.tolist())   # 第一页收盘价转为list
        else:
            print('error:', data)

    #     while page_req_key != None:  # 请求后面的所有结果
    #         print('*************************************')
    #         ret, data, page_req_key = quote_ctx.request_history_kline('HK.00700', start='2019-09-11', end='2020-09-18', max_count=500, ktype = KLType.K_5M, page_req_key=page_req_key) # 请求翻页后的数据
    #         if ret == RET_OK:
    #             print("==========WHILE==========")
    #             print(data)
    #             df_list.append(data)
    #         else:
    #             print('error:', data)
    # print('All pages are finished!')
    all_df = pd.concat(all_list)


    """
    Calculating Signals
    """

    stock_list = []
    date_list = []
    signal_list = []
    len_ = 1
    date_col = "time_key"
    price_col = "close"
    ticker_col = "code"
    signal_df = pd.DataFrame()
    model_freq = "1D"
    model_name = "Macd_Updown_Signals"
    total_number = len(ticker_list)


    for ticker in ticker_list:
        print ("Calculating Signals for No.{} : {}".format(len_, ticker))
        data = all_df[all_df['code']==ticker]
        try:
            data_macd = MACD(data, price_col, 12, 26)
            ticker, last_date, last_signal = macd_updown_signals(data_macd, "MACD", date_col, ticker_col)
            stock_list.append(ticker)
            date_list.append(last_date)
            signal_list.append(last_signal)
        except Exception as e:
            print(e)
        len_+= 1
    signal_df['Ticker'] = stock_list
    signal_df['Last_Timestamp'] = date_list
    signal_df['Signal'] = signal_list
    signal_df['model_name'] = model_name
    signal_df['model_freq'] = model_freq
    print("All Done!")
    return signal_df

In [100]:
signal_df_1D_macd = freq_1D_macd_up_down("2020-01-01", end_date)

Calculating Signals for No.1 : HK.09988
Calculating Signals for No.2 : HK.06618
Calculating Signals for No.3 : HK.03888
Calculating Signals for No.4 : HK.00418
Calculating Signals for No.5 : HK.09999
Calculating Signals for No.6 : HK.01909
Calculating Signals for No.7 : HK.01070
Calculating Signals for No.8 : HK.06969
Calculating Signals for No.9 : HK.00241
Calculating Signals for No.10 : HK.01137
Calculating Signals for No.11 : HK.06055
Calculating Signals for No.12 : HK.06862
Calculating Signals for No.13 : HK.00700
Calculating Signals for No.14 : HK.02051
Calculating Signals for No.15 : HK.06993
Calculating Signals for No.16 : HK.06618
Calculating Signals for No.17 : HK.09992
Calculating Signals for No.18 : HK.03690
Calculating Signals for No.19 : HK.01810
Calculating Signals for No.20 : HK.01024
list index out of range
Calculating Signals for No.21 : HK.01119
Calculating Signals for No.22 : HK.08029
Calculating Signals for No.23 : HK.06688
index -1 is out of bounds for axis 0 with 

<ipython-input-98-b8490d96d521>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DIFF"] = diff
<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

Calculating Signals for No.27 : HK.01610
Calculating Signals for No.28 : HK.02400
All Done!


<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MACD"] = macd
<ipython-input-98-b8490d96d521>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


In [101]:
signal_df_1D_macd

,Ticker,Last_Timestamp,Signal,model_name,model_freq
0,HK.09988,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
1,HK.06618,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
2,HK.03888,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
3,HK.00418,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
4,HK.09999,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
5,HK.01909,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
6,HK.01070,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
7,HK.06969,2021-02-24 00:00:00,SELL,Macd_Updown_Signals,1D
8,HK.00241,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
9,HK.01137,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D


In [102]:
signal_df_15m_macd = freq_15M_macd_up_down("2021-02-10", end_date)

Calculating Signals for No.1 : HK.09988
Calculating Signals for No.2 : HK.06618
Calculating Signals for No.3 : HK.03888
Calculating Signals for No.4 : HK.00418
Calculating Signals for No.5 : HK.09999
Calculating Signals for No.6 : HK.01909
Calculating Signals for No.7 : HK.01070
Calculating Signals for No.8 : HK.06969
Calculating Signals for No.9 : HK.00241
Calculating Signals for No.10 : HK.01137
Calculating Signals for No.11 : HK.06055
Calculating Signals for No.12 : HK.06862
Calculating Signals for No.13 : HK.00700
Calculating Signals for No.14 : HK.02051
Calculating Signals for No.15 : HK.06993
Calculating Signals for No.16 : HK.06618
Calculating Signals for No.17 : HK.09992
Calculating Signals for No.18 : HK.03690
Calculating Signals for No.19 : HK.01810
Calculating Signals for No.20 : HK.01024
Calculating Signals for No.21 : HK.01119
Calculating Signals for No.22 : HK.08029
Calculating Signals for No.23 : HK.06688
list index out of range
Calculating Signals for No.24 : HK.00493
C

<ipython-input-98-b8490d96d521>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DIFF"] = diff
<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

All Done!


In [103]:
# 加时间：
time.sleep(30)

In [104]:
signal_df_60m_macd = freq_60M_macd_up_down("2021-01-01", end_date)

Calculating Signals for No.1 : HK.09988
Calculating Signals for No.2 : HK.06618
Calculating Signals for No.3 : HK.03888
Calculating Signals for No.4 : HK.00418
Calculating Signals for No.5 : HK.09999
Calculating Signals for No.6 : HK.01909
Calculating Signals for No.7 : HK.01070
Calculating Signals for No.8 : HK.06969
Calculating Signals for No.9 : HK.00241
Calculating Signals for No.10 : HK.01137
Calculating Signals for No.11 : HK.06055
Calculating Signals for No.12 : HK.06862
Calculating Signals for No.13 : HK.00700
Calculating Signals for No.14 : HK.02051
Calculating Signals for No.15 : HK.06993
Calculating Signals for No.16 : HK.06618
Calculating Signals for No.17 : HK.09992
Calculating Signals for No.18 : HK.03690
Calculating Signals for No.19 : HK.01810
Calculating Signals for No.20 : HK.01024

<ipython-input-98-b8490d96d521>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DIFF"] = diff
<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers


Calculating Signals for No.21 : HK.01119
Calculating Signals for No.22 : HK.08029
Calculating Signals for No.23 : HK.06688
list index out of range
Calculating Signals for No.24 : HK.00493
Calculating Signals for No.25 : HK.00175
Calculating Signals for No.26 : HK.06969
Calculating Signals for No.27 : HK.01610
Calculating Signals for No.28 : HK.02400
All Done!


<ipython-input-98-b8490d96d521>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DIFF"] = diff
<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [105]:
# 加时间：
time.sleep(30)

In [106]:
signal_df_1D_macd = freq_1D_macd_up_down("2020-01-01", end_date)

Calculating Signals for No.1 : HK.09988
Calculating Signals for No.2 : HK.06618
Calculating Signals for No.3 : HK.03888
Calculating Signals for No.4 : HK.00418
Calculating Signals for No.5 : HK.09999
Calculating Signals for No.6 : HK.01909
Calculating Signals for No.7 : HK.01070
Calculating Signals for No.8 : HK.06969
Calculating Signals for No.9 : HK.00241
Calculating Signals for No.10 : HK.01137
Calculating Signals for No.11 : HK.06055
Calculating Signals for No.12 : HK.06862
Calculating Signals for No.13 : HK.00700
Calculating Signals for No.14 : HK.02051
Calculating Signals for No.15 : HK.06993


<ipython-input-98-b8490d96d521>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DIFF"] = diff
<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

Calculating Signals for No.16 : HK.06618
Calculating Signals for No.17 : HK.09992
Calculating Signals for No.18 : HK.03690
Calculating Signals for No.19 : HK.01810
Calculating Signals for No.20 : HK.01024
list index out of range
Calculating Signals for No.21 : HK.01119
Calculating Signals for No.22 : HK.08029
Calculating Signals for No.23 : HK.06688
index -1 is out of bounds for axis 0 with size 0
Calculating Signals for No.24 : HK.00493
Calculating Signals for No.25 : HK.00175
Calculating Signals for No.26 : HK.06969
Calculating Signals for No.27 : HK.01610
Calculating Signals for No.28 : HK.02400
All Done!


<ipython-input-98-b8490d96d521>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DIFF"] = diff
<ipython-input-98-b8490d96d521>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DEA"] = dea
<ipython-input-98-b8490d96d521>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [107]:
print("length of all tickers", len(ticker_list))
print("length of 15m tickers:", len(signal_df_15m_macd))
print("length of 60m tickers:", len(signal_df_60m_macd))
print("length of 1D tickers:", len(signal_df_1D_macd))

length of all tickers 28
length of 15m tickers: 27
length of 60m tickers: 27
length of 1D tickers: 26


In [108]:
merged_all = signal_df_60m_macd.merge(signal_df_1D_macd, on = "Ticker", how = "outer")
merged_all = signal_df_15m_macd.merge(merged_all, on = "Ticker", how = "outer")

merged_all_buy_holdlong_lt = merged_all[((merged_all['Signal_x']=="BUY")&(merged_all['Signal_y']=='BUY')) |\
                             ((merged_all['Signal_x']=='BUY')&(merged_all['Signal_y']=='HOLD LONG')) |
                             ((merged_all['Signal_x']=='HOLD LONG')&(merged_all['Signal_y']=='HOLD LONG'))]
merged_all_buy_holdlong_st = merged_all_buy_holdlong_lt[(merged_all_buy_holdlong_lt["Signal"]=="BUY")|(merged_all_buy_holdlong_lt["Signal"]=="HOLD LONG")]
merged_all_buy_holdlong_st.drop_duplicates(inplace = True)

,Ticker,Last_Timestamp,Signal,model_name,model_freq,Last_Timestamp_x,Signal_x,model_name_x,model_freq_x,Last_Timestamp_y,Signal_y,model_name_y,model_freq_y


In [109]:
merged_all.drop_duplicates()

,Ticker,Last_Timestamp,Signal,model_name,model_freq,Last_Timestamp_x,Signal_x,model_name_x,model_freq_x,Last_Timestamp_y,Signal_y,model_name_y,model_freq_y
0,HK.09988,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,SELL,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
1,HK.06618,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,SELL,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
9,HK.03888,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,HOLD SHORT,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
10,HK.00418,2021-02-24 11:45:00,HOLD LONG,Macd_Updown_Signals,15M,2021-02-24 11:30:00,BUY,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
11,HK.09999,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,HOLD SHORT,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
12,HK.01909,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,HOLD SHORT,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
13,HK.01070,2021-02-24 11:45:00,HOLD SHORT,Macd_Updown_Signals,15M,2021-02-24 11:30:00,SELL,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
14,HK.06969,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,HOLD SHORT,Macd_Updown_Signals,60M,2021-02-24 00:00:00,SELL,Macd_Updown_Signals,1D
22,HK.00241,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,HOLD SHORT,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D
23,HK.01137,2021-02-24 11:45:00,BUY,Macd_Updown_Signals,15M,2021-02-24 11:30:00,HOLD SHORT,Macd_Updown_Signals,60M,2021-02-24 00:00:00,HOLD SHORT,Macd_Updown_Signals,1D


In [110]:
merged_all.drop_duplicates()[['Ticker', 'Last_Timestamp', 'model_name','Signal','model_freq',
                              'Signal_x', 'model_freq_x',
                              'Signal_y', 'model_freq_y']]

,Ticker,Last_Timestamp,model_name,Signal,model_freq,Signal_x,model_freq_x,Signal_y,model_freq_y
0,HK.09988,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,SELL,60M,HOLD SHORT,1D
1,HK.06618,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,SELL,60M,HOLD SHORT,1D
9,HK.03888,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,HOLD SHORT,60M,HOLD SHORT,1D
10,HK.00418,2021-02-24 11:45:00,Macd_Updown_Signals,HOLD LONG,15M,BUY,60M,HOLD SHORT,1D
11,HK.09999,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,HOLD SHORT,60M,HOLD SHORT,1D
12,HK.01909,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,HOLD SHORT,60M,HOLD SHORT,1D
13,HK.01070,2021-02-24 11:45:00,Macd_Updown_Signals,HOLD SHORT,15M,SELL,60M,HOLD SHORT,1D
14,HK.06969,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,HOLD SHORT,60M,SELL,1D
22,HK.00241,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,HOLD SHORT,60M,HOLD SHORT,1D
23,HK.01137,2021-02-24 11:45:00,Macd_Updown_Signals,BUY,15M,HOLD SHORT,60M,HOLD SHORT,1D


In [111]:
merged_all_buy_holdlong_lt

,Ticker,Last_Timestamp,Signal,model_name,model_freq,Last_Timestamp_x,Signal_x,model_name_x,model_freq_x,Last_Timestamp_y,Signal_y,model_name_y,model_freq_y


In [112]:
merged_all_buy_holdlong_st

,Ticker,Last_Timestamp,Signal,model_name,model_freq,Last_Timestamp_x,Signal_x,model_name_x,model_freq_x,Last_Timestamp_y,Signal_y,model_name_y,model_freq_y


2021-02-25 09:25:23,866 [open_context_base.py] on_error:366: Connect error: conn_id=5; err=[WinError 10054] 远程主机强迫关闭了一个现有的连接。;
2021-02-25 09:25:25,277 [open_context_base.py] _wait_reconnect:472: Wait reconnect in 8 seconds: host=127.0.0.1; port=11111;
2021-02-25 09:25:26,170 [network_manager.py] work:366: Close: conn_id=5
2021-02-25 09:25:33,473 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2021-02-25 09:25:33,734 [open_context_base.py] on_connected:344: Connected : conn_id=7; 
2021-02-25 09:25:34,620 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=7; info={'server_version': 500, 'login_user_id': 22320258, 'conn_id': 6770514028531630601, 'conn_key': '13E41EE7CF9294F6', 'conn_iv': '6071B6534569E1F3', 'keep_alive_interval': 10, 'is_encrypt': False};
